In [1]:
# Установка googletrans
!pip install googletrans==4.0.0rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 938.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.0 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=193c1bda7464083ca83ce952cfc76817a4963d8c5d64ea97ede1b8b2ab20cd96
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfr

# 1. Загрузка kaggle.json

Перед запуском кода:

1.   Перейдите на [страницу настроек Kaggle](https://www.kaggle.com/settings)
2.   Нажмите **Create New Token**
3.   Скачайте kaggle.json
4.   **Загрузите файл в Colab**, выполнив ячейку ниже:

In [2]:
from google.colab import files

uploaded = files.upload()  # Загрузите kaggle.json вручную

Saving kaggle.json to kaggle.json


# 2. Установка и настройка Kaggle API в Colab

In [3]:
!pip install -q kaggle

import os
import zipfile

# Создание папки и перемещение файла
os.makedirs("/root/.kaggle", exist_ok=True)
with open("/root/.kaggle/kaggle.json", "w") as f:
    f.write(uploaded['kaggle.json'].decode("utf-8"))

os.chmod("/root/.kaggle/kaggle.json", 600)

# 3. Скачивание и объединение датасетов

In [4]:
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi

# Указываем ID датасетов
datasets = [
    "kazanova/sentiment140",
    "bhavikjikadara/tweets-dataset",
    "edqian/twitter-climate-change-sentiment-dataset",
    "joseguzman/climate-sentiment-in-twitter",
    "die9origephit/climate-change-tweets"
]

download_dir = "kaggle_data"
os.makedirs(download_dir, exist_ok=True)

# Инициализация Kaggle API
api = KaggleApi()
api.authenticate()

for dataset in datasets:
    try:
        print(f"Скачиваю: {dataset}")
        api.dataset_download_files(dataset, path=download_dir, unzip=True)
    except Exception as e:
        print(f"Ошибка при скачивании {dataset}: {e}")

Скачиваю: kazanova/sentiment140
Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
Скачиваю: bhavikjikadara/tweets-dataset
Dataset URL: https://www.kaggle.com/datasets/bhavikjikadara/tweets-dataset
Скачиваю: edqian/twitter-climate-change-sentiment-dataset
Dataset URL: https://www.kaggle.com/datasets/edqian/twitter-climate-change-sentiment-dataset
Скачиваю: joseguzman/climate-sentiment-in-twitter
Dataset URL: https://www.kaggle.com/datasets/joseguzman/climate-sentiment-in-twitter
Скачиваю: die9origephit/climate-change-tweets
Dataset URL: https://www.kaggle.com/datasets/die9origephit/climate-change-tweets


In [5]:
!pip install -q chardet

import chardet

data_dir = "kaggle_data"
all_texts = []

# Возможные названия текстовых столбцов
text_column_candidates = ['text', 'tweet', 'content', 'message', 'Text', 'Tweet', 'Content', 'Message']

# Функция для определения кодировки и чтения CSV
def read_csv_with_fallback(path, candidates):
    try:
        with open(path, 'rb') as f:
            raw_data = f.read(10000)
        guess = chardet.detect(raw_data)
        encodings_to_try = [guess['encoding'], 'utf-8', 'latin1', 'ISO-8859-1']

        for encoding in encodings_to_try:
            try:
                # Пробуем сначала с заголовками
                df = pd.read_csv(path, lineterminator='\n', encoding=encoding, low_memory=False)
                # Очищаем имена столбцов от лишних символов
                df.columns = [col.strip().replace('\r', '').replace('\n', '') for col in df.columns]

                print(f"Пробую файл: {os.path.basename(path)} (кодировка: {encoding})")
                print(f"Заголовки столбцов: {list(df.columns)}")

                for col in df.columns:
                    if col.lower() in [c.lower() for c in candidates]:
                        df_text = df[[col]].copy()
                        df_text.columns = ['text']
                        print(f"Добавлено из {os.path.basename(path)} (столбец: {col}, кодировка: {encoding})")
                        return df_text

                # Спецобработка для training.1600000.processed.noemoticon.csv
                if df.shape[1] >= 6 and "noemoticon" in os.path.basename(path).lower():
                    df_text = df.iloc[:, -1].copy()
                    df_text = pd.DataFrame(df_text)
                    df_text.columns = ['text']
                    print(f"Добавлено из {os.path.basename(path)} (столбец: последний, кодировка: {encoding})")
                    return df_text

                print(f"Пропущено {os.path.basename(path)} — текстовый столбец не найден.")
                return None

            except Exception:
                continue

    except Exception as e:
        print(f"Ошибка при открытии {os.path.basename(path)}: {e}")

    print(f"Не удалось прочитать файл: {os.path.basename(path)}")
    return None

# Поиск и обработка CSV-файлов
for root, _, files in os.walk(data_dir):
    for file in files:
        if file.endswith(".csv"):
            path = os.path.join(root, file)
            df_text = read_csv_with_fallback(path, text_column_candidates)
            if df_text is not None:
                all_texts.append(df_text)

# Объединение всех текстов в один DataFrame
combined_df = pd.concat(all_texts, ignore_index=True)
combined_df.drop_duplicates(inplace=True)
combined_df.dropna(subset=['text'], inplace=True)

# Итоговая статистика и сохранение
print(f"\nВсего твитов после объединения: {len(combined_df)}")
combined_df.to_csv("combined_tweets.csv", index=False)

Пробую файл: tweets.csv (кодировка: latin1)
Заголовки столбцов: ['Target', 'ID', 'Date', 'flag', 'User', 'Text']
Добавлено из tweets.csv (столбец: Text, кодировка: latin1)
Пробую файл: Climate_twitter.csv (кодировка: utf-8)
Заголовки столбцов: ['id', 'date', 'retweets', 'source', 'author', 'likes', 'text', 'twitter_name', 'location', 'verified', 'followers', 'friends', 'polarity', 'subjectivity']
Добавлено из Climate_twitter.csv (столбец: text, кодировка: utf-8)
Пробую файл: training.1600000.processed.noemoticon.csv (кодировка: latin1)
Заголовки столбцов: ['0', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY', '_TheSpecialOne_', "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"]
Добавлено из training.1600000.processed.noemoticon.csv (столбец: последний, кодировка: latin1)
Пробую файл: Climate change_2022-1-17_2022-7-19.csv (кодировка: utf-8)
Заголовки столбцов: ['UserScreenName', 'UserName', 'Timestamp', 'Text'

# 4. Загрузка ключевых слов на русском языке

In [6]:
!pip install gdown chardet

import gdown
import chardet

# Скачивание файла с ключевыми словами с Google Диска
file_url = 'https://drive.google.com/uc?id=***'  # Укажите ID файла из ссылки
# (Мой диск: напротив keywords.txt 3 точки - Открыть доступ - Все, у кого есть ссылка - Копировать ссылку
# - Вставить в код все, что находится в ссылке вместо *** https://drive.google.com/file/d/***/view?usp=sharing)
file_name = 'keywords.txt'

def download_keywords_file(url, output):
    print("Загружаем файл ключевых слов...")
    gdown.download(url, output, quiet=False)
    print(f"Файл сохранён как {output}")

def load_keywords(file_name):
    print("Читаем ключевые слова из файла...")
    with open(file_name, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        print(f"Определённая кодировка файла: {encoding}")

    try:
        content = raw_data.decode(encoding)
        # Разделение по строкам
        keywords = [line.strip() for line in content.splitlines() if line.strip()]
        return keywords
    except Exception as e:
        print(f"Ошибка при чтении файла: {e}")
        return []

    # Загрузка ключевых слов
download_keywords_file(file_url, file_name)
keywords = load_keywords(file_name)

Загружаем файл ключевых слов...


Downloading...
From: https://drive.google.com/uc?id=1bsFU4q_jPQ-J0V1M4zB1OJZona2-ppRE
To: /content/keywords.txt
100%|██████████| 19.0k/19.0k [00:00<00:00, 20.4MB/s]

Файл сохранён как keywords.txt
Читаем ключевые слова из файла...
Определённая кодировка файла: utf-8


# 5. Перевод ключевых слов на английский

In [7]:
from googletrans import Translator
import time

# Функция для перевода ключевых слов
def translate_keywords(keywords, max_retries=3):
    translator = Translator()
    translated_keywords = []
    failed_translations = []

    for keyword in keywords:
        keyword = keyword.strip()
        if not keyword:
            translated_keywords.append('')
            continue

        attempt = 0
        translated = None

        while attempt < max_retries:
            try:
                result = translator.translate(keyword, src='ru', dest='en')
                translated = result.text
                break
            except Exception as e:
                print(f"Попытка {attempt + 1}: ошибка при переводе слова '{keyword}': {e}")
                time.sleep(1)
                attempt += 1

        if translated:
            translated_keywords.append(translated)
        else:
            failed_translations.append(keyword)
            translated_keywords.append(None)

    return translated_keywords, failed_translations

# Переводим ключевые слова
translated_keywords, failed = translate_keywords(keywords)

# Обработка неудачных переводов
for i, keyword in enumerate(keywords):
    if translated_keywords[i] is None:
        user_input = input(f"Не удалось перевести '{keyword}'. Введите перевод вручную: ")
        translated_keywords[i] = user_input

# Выводим переведенные ключевые слова
print("\nПереведённые ключевые слова:")
print(translated_keywords)

# Сохраняем переведенные ключевые слова в новый файл
with open('translated_keywords.txt', 'w', encoding='utf-8') as f:
    for keyword in translated_keywords:
        f.write(f"{keyword}\n")

print("Переведённые ключевые слова сохранены в 'translated_keywords.txt'")


Переведённые ключевые слова:
['Cape Schmidt', 'Northern Polyus-27', 'Northern Forum', 'Lake Taimyr', 'Gaiberg island', 'Northern Sea Route Logistics', 'Arctic research hospital', 'Arctic logistics', 'Zhokhov Island', 'Providence Bay', 'Ecology of the Arctic', 'Moosts Island', 'Seal', 'Polar culture', 'Small Taimyr', 'The island is white', 'Arctic project', 'Ward-Hant Island', 'Ust-Odlenik', 'Hornsunn', 'Arctic Circle', 'Geophysical station of V.V. Khodov flask', 'Glacier', 'Russian harbor', 'Co -ups of ice', 'Oil production', 'Lena-Nerdenscheld', 'Russian mouth', 'Polar birds', 'Soviet Arctic', 'The island of four -stage', 'A scale of ice of ice', 'Ikaluite research center', 'North Polyus-22', 'The island of Vilkitsky', 'Climate anomalies', 'Hydrology of the Arctic', 'The global atmosphere observatory', 'Iznov Izvestia CEC', 'Flashline', 'Ust-Odlenek', 'Cape Boor Hay', 'The island of Samoilovsky', 'New Earth', 'Polar night', 'Arctic oil production', 'Sea ecosystems', 'North Pole-18', 

# 6. Отбор твитов с ключевыми словами

In [10]:
!pip install tqdm
from tqdm import tqdm

import pandas as pd
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet, stopwords

# Подключение Google Диска
from google.colab import drive
drive.mount('/content/drive')

# Путь для сохранения файла
save_path = "/content/drive/MyDrive/TextScope/selected_tweets.csv"

# Загрузка ресурсов NLTK (исправлены названия)
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Читаем переведённые ключевые слова из файла
with open('translated_keywords.txt', 'r', encoding='utf-8') as f:
    translated_keywords = [line.strip().lower() for line in f if line.strip()]

# Мягкая очистка текста (ссылки, упоминания, хэштеги и т.д.)
def soft_preprocess(text):
    t = str(text).lower()
    t = re.sub(r"http\S+|www\S+|t\.me/\S+", "", t)  # ссылки
    t = re.sub(r"@\w+", "", t)  # упоминания
    t = re.sub(r"#\w+", "", t)  # хэштеги
    t = re.sub(r"[^\w\s]", "", t)  # все кроме слов и пробелов
    t = re.sub(r"\d+", "", t)  # числа
    t = re.sub(r"\s+", " ", t).strip()  # лишние пробелы
    return t

# Получение POS-тегов для WordNet
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# Полная лемматизация текста + удаление стоп-слов и коротких слов
def full_preprocess(text):
    tokens = word_tokenize(text)
    tagged = pos_tag(tokens)
    lemmas = [lemmatizer.lemmatize(w, get_wordnet_pos(t)) for w, t in tagged]
    lemmas = [w for w in lemmas if w not in stop_words and len(w) > 2]
    return lemmas

# Лемматизация ключей
lemmatized_keywords = set(full_preprocess(" ".join(translated_keywords)))

# Загрузка твитов
df = pd.read_csv("combined_tweets.csv")

selected_rows = 0

# Создаём файл и записываем заголовок
with open(save_path, 'w', encoding='utf-8') as f_out:
    f_out.write("text,cleaned_text\n")  # заголовки

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Обработка твитов"):
        original_text = row['text']
        cleaned_text = soft_preprocess(original_text)
        lemmas = set(full_preprocess(cleaned_text))

        if lemmas & lemmatized_keywords:  # если есть пересечение с ключами
            selected_rows += 1
            final_text = " ".join(lemmas)
            # Экранируем кавычки для CSV
            original = original_text.replace('"', '""')
            cleaned = final_text.replace('"', '""')
            f_out.write(f'"{original}","{cleaned}"\n')

print(f"Сохранение завершено: {save_path}")
print(f"Отобрано строк: {selected_rows} из {len(df)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Лемматизированные ключевые слова: {'indigenous', 'popov', 'hay', 'alomar', 'dixon', 'aeroarctic', 'mouth', 'hydrometeorological', 'trade', 'field', 'conservation', 'ion', 'polyus-22', 'nku', 'change', 'geology', 'sunny', 'pure', 'exploration', 'valkarkai', 'solitude', 'verenskiold', 'archipelago', 'rosprirodnadzor', 'anyui', 'cryology', 'gaiberg', 'sagyllah-ara', 'komsomolskaya', 'lake', 'fact', 'yurika', 'anomaly', 'ust-odlenik', 'sopular', 'forecast', 'baranova', 'polyus-29', 'quiet', 'polyus-26', 'atlantic', 'warming', 'transfiguration', 'canadian', 'extreme', 'schelph', 'automatic', 'pole-11', 'sea', 'polyus-25', 'frequency', 'shalaurov', 'belarusian', 'seyakha', 'chaun', 'ugra', 'revolution', 'leskino', 'aleutian', 'fluctuation', 'wrangel', 'rauchea', 'geopolitics', 'matochkin', 'four', 'politics', 'explorer', 'vavilov', 'cover', 'climatological', 'polyus-5', 'geophysical', 'vatutin', 'pyasin', 'pama', 'hospital', 'island', 'sverdrupe', 'kolguev', 'polyus-20', 'tundra', 'alexandra

Обработка твитов: 100%|██████████| 2667902/2667902 [46:55<00:00, 947.48it/s]

Сохранение завершено: /content/drive/MyDrive/TextScope/selected_tweets.csv
Отобрано строк: 787576 из 2667902


# 7. Тематическое моделирование BERTopic

In [ ]:
# Среда выполнения → Сменить среду выполнения → Графический процессор T4

# GPU ускоряет обработку моделей и данных, делая выполнение кода в разы быстрее по сравнению с обычным процессором

In [1]:
# Подключение Google Диска
from google.colab import drive
drive.mount('/content/drive')

# Установка необходимых библиотек
!pip install --upgrade --force-reinstall numpy
!pip install -U scikit-learn bertopic

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 103.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.3.0 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.0 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.0 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 131.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 116.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Среда выполнения → Перезапустить сеанс

In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from umap import UMAP
from hdbscan import HDBSCAN
import os
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

#path = {"twitter": "/content/drive/MyDrive/TextScope/selected_tweets.csv"}
path = "/content/drive/MyDrive/TextScope/selected_tweets.csv"

plots_dir = "/content/drive/MyDrive/TextScope/plots"
os.makedirs(plots_dir, exist_ok=True)

In [23]:
def bertopic_pipeline(df, text_column, out_path, source, batch_size=5000, sample_size=50000):
    print(f"Запускаем BERTopic для {source} с batch_size={batch_size} и sample_size={sample_size}")

    # Удаляем пустые строки перед выборкой
    df_non_empty = df[df[text_column].astype(str).str.strip() != ""]

    if len(df_non_empty) == 0:
        raise ValueError("В df нет строк с непустым текстом")

    if sample_size is not None and len(df_non_empty) > sample_size:
        df_sampled = df_non_empty.sample(n=sample_size, random_state=42)
    else:
        df_sampled = df_non_empty.copy()

    docs = df_sampled[text_column].astype(str).tolist()

    if len(docs) == 0:
        raise ValueError("Нет доступных документов после предобработки")

    device = "cuda" if __import__('torch').cuda.is_available() else "cpu"
    embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2", device=device)

    all_embeddings = []

    print("Вычисляем эмбеддинги батчами...")
    for i in tqdm(range(0, len(docs), batch_size)):
        batch_docs = docs[i:i+batch_size]
        if len(batch_docs) == 0:
            continue
        batch_embeddings = embedder.encode(batch_docs, show_progress_bar=False)
        all_embeddings.extend(batch_embeddings)

    if len(all_embeddings) == 0:
        raise ValueError("Эмбеддинги пусты. Проверьте содержимое docs после предобработки.")

    all_embeddings = np.vstack(all_embeddings)
    print("Эмбеддинги готовы. Обучаем BERTopic...")

    model = BERTopic(
        embedding_model=embedder,
        nr_topics="auto",
        umap_model=UMAP(n_neighbors=5, n_components=10, min_dist=0.3, metric='cosine'),
        hdbscan_model=HDBSCAN(min_cluster_size=30, min_samples=5, metric='euclidean', cluster_selection_method='leaf'),
        calculate_probabilities=False,
        verbose=False
    )
    model.fit(docs, all_embeddings)

    print("Визуализация тем (интерактивная)...")
    fig = model.visualize_topics()
    display(HTML(fig.to_html(include_plotlyjs='cdn')))

    # Сохраняем первичную визуализацию
    plots_dir = "/content/drive/MyDrive/TextScope/plots"
    os.makedirs(plots_dir, exist_ok=True)
    html_path_initial = os.path.join(plots_dir, f"{source}_bertopic_initial.html")
    fig.write_html(html_path_initial)
    print(f"Первичная визуализация BERTopic сохранена в {html_path_initial}")

    n_topics_widget = widgets.IntText(value=10, description='Число тем:', min=2)
    button = widgets.Button(description="Применить редукцию")
    output = widgets.Output()
    progress = widgets.Label(value="Ожидание выбора пользователем...")

    def on_button_clicked(b):
        with output:
            clear_output(wait=True)
            n_topics = n_topics_widget.value
            current_num_topics = len(set(model.topics_)) - (1 if -1 in model.topics_ else 0)
            print(f"Количество уникальных тем в модели: {current_num_topics}")

            if n_topics >= current_num_topics:
                print(f"Ошибка: выбрано {n_topics} тем, но в модели только {current_num_topics}. Установите значение меньше.")
                progress.value = "Ошибка выбора числа тем."
                return

            print(f"Выбрано {n_topics} тем.")
            progress.value = "Редукция тем..."

            # Редукция тем
            reduced_model = model.reduce_topics(docs, nr_topics=n_topics)

            # Получаем новые топики через transform
            new_topics, _ = reduced_model.transform(docs)

            progress.value = "Редукция завершена. Создание визуализации..."

            fig_reduced = reduced_model.visualize_topics()
            display(HTML(fig_reduced.to_html(include_plotlyjs='cdn')))

            plot_path = os.path.join(plots_dir, f"{source}_reduced_topics_{n_topics}.html")
            with open(plot_path, "w", encoding="utf-8") as f:
                f.write(fig_reduced.to_html(include_plotlyjs='cdn'))

            progress.value = "Сохранение CSV и модели..."

            # Используем new_topics и df_sampled!
            df_sampled["topic"] = new_topics
            df_sampled.to_csv(out_path, index=False, quoting=1)

            model_path = out_path.replace(".csv", f"_{source}_model_{n_topics}")
            reduced_model.save(model_path)

            progress.value = "Файлы сохранены."
            print(f"Результаты сохранены:\nCSV: {out_path}\nМодель: {model_path}\nГрафик: {plot_path}")

    button.on_click(on_button_clicked)
    display(n_topics_widget, button, progress, output)

In [25]:
def bertopic_pipeline(df, text_column, out_path, source, batch_size=5000, sample_size=50000):
    print(f"Запускаем BERTopic для {source} с batch_size={batch_size} и sample_size={sample_size}")

    # Удаляем пустые строки перед выборкой
    df_non_empty = df[df[text_column].astype(str).str.strip() != ""]

    if len(df_non_empty) == 0:
        raise ValueError("В df нет строк с непустым текстом")

    if sample_size is not None and len(df_non_empty) > sample_size:
        df_sampled = df_non_empty.sample(n=sample_size, random_state=42)
    else:
        df_sampled = df_non_empty.copy()

    docs = df_sampled[text_column].astype(str).tolist()

    if len(docs) == 0:
        raise ValueError("Нет доступных документов после предобработки")

    device = "cuda" if __import__('torch').cuda.is_available() else "cpu"
    embedder = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2", device=device)

    all_embeddings = []

    print("Вычисляем эмбеддинги батчами...")
    for i in tqdm(range(0, len(docs), batch_size)):
        batch_docs = docs[i:i+batch_size]
        if len(batch_docs) == 0:
            continue
        batch_embeddings = embedder.encode(batch_docs, show_progress_bar=False)
        all_embeddings.extend(batch_embeddings)

    if len(all_embeddings) == 0:
        raise ValueError("Эмбеддинги пусты. Проверьте содержимое docs после предобработки.")

    all_embeddings = np.vstack(all_embeddings)
    print("Эмбеддинги готовы. Обучаем BERTopic...")

    model = BERTopic(
        embedding_model=embedder,
        nr_topics="auto",
        umap_model=UMAP(n_neighbors=5, n_components=10, min_dist=0.3, metric='cosine'),
        hdbscan_model=HDBSCAN(min_cluster_size=30, min_samples=5, metric='euclidean', cluster_selection_method='leaf'),
        calculate_probabilities=False,
        verbose=False
    )
    model.fit(docs, all_embeddings)

    print("Визуализация тем (интерактивная)...")
    fig = model.visualize_topics()
    display(HTML(fig.to_html(include_plotlyjs='cdn')))

    plots_dir = "/content/drive/MyDrive/TextScope/plots"
    os.makedirs(plots_dir, exist_ok=True)
    html_path_initial = os.path.join(plots_dir, f"{source}_bertopic_initial.html")
    fig.write_html(html_path_initial)
    print(f"Первичная визуализация BERTopic сохранена в {html_path_initial}")

    n_topics_widget = widgets.IntText(value=10, description='Число тем:', min=2)
    button = widgets.Button(description="Применить редукцию")
    output = widgets.Output()
    progress = widgets.Label(value="Ожидание выбора пользователем...")

    def on_button_clicked(b):
        with output:
            clear_output(wait=True)
            n_topics = n_topics_widget.value
            current_num_topics = len(set(model.topics_)) - (1 if -1 in model.topics_ else 0)
            print(f"Количество уникальных тем в модели: {current_num_topics}")

            if n_topics >= current_num_topics:
                print(f"Ошибка: выбрано {n_topics} тем, но в модели только {current_num_topics}. Установите значение меньше.")
                progress.value = "Ошибка выбора числа тем."
                return

            print(f"Выбрано {n_topics} тем.")
            progress.value = "Редукция тем..."

            reduced_model = model.reduce_topics(docs, nr_topics=n_topics)

            progress.value = "Редукция завершена. Создание визуализации..."

            fig_reduced = reduced_model.visualize_topics()
            display(HTML(fig_reduced.to_html(include_plotlyjs='cdn')))

            plot_path = os.path.join(plots_dir, f"{source}_reduced_topics_{n_topics}.html")
            with open(plot_path, "w", encoding="utf-8") as f:
                f.write(fig_reduced.to_html(include_plotlyjs='cdn'))

            progress.value = "Сохранение CSV и модели..."

            # Используем напрямую reduced_model.topics_
            df_sampled["topic"] = reduced_model.topics_
            df_sampled.to_csv(out_path, index=False, quoting=1)

            model_path = out_path.replace(".csv", f"_{source}_model_{n_topics}")
            reduced_model.save(model_path)

            progress.value = "Файлы сохранены."
            print(f"Результаты сохранены:\nCSV: {out_path}\nМодель: {model_path}\nГрафик: {plot_path}")

    button.on_click(on_button_clicked)
    display(n_topics_widget, button, progress, output)

In [26]:
key = 'twitter'
#path = path[key]
out_path = path.replace(".csv", "_topics.csv")

if os.path.exists(out_path):
    print(f"{out_path} уже существует — пропускаем.")
else:
    print(f"Обрабатываем файл: {path}")
    df = pd.read_csv(path)
    bertopic_pipeline(df, text_column="cleaned_text", out_path=out_path, source=key, batch_size=5000, sample_size=50000)

Обрабатываем файл: /content/drive/MyDrive/TextScope/selected_tweets.csv
Запускаем BERTopic для twitter с batch_size=5000 и sample_size=50000
Вычисляем эмбеддинги батчами...


100%|██████████| 10/10 [00:20<00:00,  2.03s/it]


Эмбеддинги готовы. Обучаем BERTopic...
Визуализация тем (интерактивная)...


Первичная визуализация BERTopic сохранена в /content/drive/MyDrive/TextScope/plots/twitter_bertopic_initial.html


IntText(value=10, description='Число тем:')

Button(description='Применить редукцию', style=ButtonStyle())

Label(value='Ожидание выбора пользователем...')

Output()

# Просмотр топ-слов по темам

In [27]:
from google.colab import drive
drive.mount('/content/drive')

!pip install bertopic

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import glob
from bertopic import BERTopic

# Пути к CSV-файлам в Google Drive
paths = {
    "twitter": "/content/drive/MyDrive/TextScope/selected_tweets_topics.csv",
}

# Функция для поиска пути к модели
def find_bertopic_model(path_csv, source):
    pattern = path_csv.replace(".csv", f"_{source}_model_*")
    matches = glob.glob(pattern)
    if matches:
        print(f"Найден путь для {source}: {matches[0]}")
        return matches[0]
    else:
        raise FileNotFoundError(f"BERTopic model for {source} not found.")

# Функция для печати топ-слов тем
def print_top_words(model_path, source, n_words=10):
    model = BERTopic.load(model_path)
    topics_info = model.get_topic_info()
    print(f"\nИсточник: {source} — список тем и ключевых слов:\n")

    for topic_id in topics_info['Topic']:
        if topic_id == -1:
            continue  # пропуск аутлайеров
        top_words = model.get_topic(topic_id)
        words = [word for word, _ in top_words[:n_words]]
        print(f"Тема {topic_id}: {', '.join(words)}")

# Основной запуск
for source, csv_path in paths.items():
    try:
        model_path = find_bertopic_model(csv_path, source)
        print_top_words(model_path, source)
    except FileNotFoundError as e:
        print(e)

Найден путь для twitter: /content/drive/MyDrive/TextScope/selected_tweets_topics_twitter_model_25

Источник: twitter — список тем и ключевых слов:

Тема 0: work, night, well, weekend, day, good, sleep, home, get, feel
Тема 1: study, exam, school, work, project, book, math, back, get, time
Тема 2: twitter, video, new, work, blog, picture, tweet, pic, youtube, email
Тема 3: cream, ice, cheese, coffee, eat, lunch, tea, dinner, pizza, drink
Тема 4: climate, change, global, warming, snow, trump, cold, warm, believe, real
Тема 5: phone, laptop, iphone, new, computer, internet, ipod, work, battery, get
Тема 6: new, moon, movie, album, dance, watch, trailer, song, music, see
Тема 7: ticket, train, flight, plane, york, station, france, sell, airport, metro
Тема 8: camp, lake, swim, game, ball, club, pool, lose, beach, win
Тема 9: headache, nose, hospital, flu, sick, hurt, sore, teeth, home, fever
Тема 10: room, shower, clean, pack, laundry, shopping, clothes, buy, new, wear
Тема 11: tire, car, 

In [ ]:
from bertopic import BERTopic

def print_twitter_top_words(model_path, n_words=10):
    model = BERTopic.load(model_path)
    topics_info = model.get_topic_info()  # DataFrame со всеми темами
    print("twitter: список тем и ключевых слов:\n")

    for topic_id in topics_info['Topic']:
        if topic_id == -1:
            continue  # -1 обычно "outliers"
        top_words = model.get_topic(topic_id)
        words = [word for word, _ in top_words[:n_words]]
        print(f"Тема {topic_id}: {', '.join(words)}")

print_twitter_top_words("/content/drive/MyDrive/TextScope/selected_tweets_topics_twitter_model_{n_topics}")

# Перевод слов в выделенных темах

In [ ]:
# Среда выполнения → Сменить среду выполнения → Графический процессор T4

# GPU ускоряет обработку моделей и данных, делая выполнение кода в разы быстрее по сравнению с обычным процессором

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [4]:
import pandas as pd
import glob
from bertopic import BERTopic

# Путь к данным и модели Twitter
twitter_csv_path = "/content/drive/MyDrive/TextScope/selected_tweets_topics.csv"
twitter_model_pattern = twitter_csv_path.replace(".csv", "_twitter_model_*")
twitter_model_path = glob.glob(twitter_model_pattern)[0]  # берем первую найденную модель

print(f"Модель Twitter найдена: {twitter_model_path}")

# Загрузка данных и модели
df_twitter = pd.read_csv(twitter_csv_path)
model_twitter = BERTopic.load(twitter_model_path)
topics_twitter = model_twitter.get_topics()

# Подготовка данных
rows = []
for topic_num, words_weights in topics_twitter.items():
    words = [w for w, _ in words_weights[:10]]  # топ-10 слов
    rows.append({
        "topic_num": topic_num,
        "words_en": ", ".join(words)
    })

df_topics_en = pd.DataFrame(rows)
output_en_path = "/content/drive/MyDrive/TextScope/twitter_topics_en.csv"
df_topics_en.to_csv(output_en_path, index=False, encoding="utf-8-sig")

print(f"Темы на английском сохранены в {output_en_path}")

Модель Twitter найдена: /content/drive/MyDrive/TextScope/selected_tweets_topics_twitter_model_25
Темы на английском сохранены в /content/drive/MyDrive/TextScope/twitter_topics_en.csv


In [ ]:
# Среда выполнения → Перезапустить сеанс

In [1]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=df1347710f9fcd236364048b157c7ccb86987782da27a1bf2db05ce7cdae0343
  Stored in directory: /root/.cache/pip/wheels/39/17/6f/66a045ea3d168826074691b4b787b8f324d3f646d755443fda
Successfully built googletrans
  Attempting uninstall: hyperframe
    Found existing installation: hyperframe 6.1.0
    Uninstalling hyperfra

In [2]:
import pandas as pd
from googletrans import Translator

input_en_path = "/content/drive/MyDrive/TextScope/twitter_topics_en.csv"
output_ru_path = "/content/drive/MyDrive/TextScope/twitter_topics_en_ru.csv"

df_topics = pd.read_csv(input_en_path)

translator = Translator()

translated_words = []

for idx, row in df_topics.iterrows():
    words_en = row["words_en"].split(", ")
    words_ru = []
    for word in words_en:
        try:
            translated = translator.translate(word, src='en', dest='ru').text
        except Exception as e:
            print(f"Ошибка перевода слова '{word}': {e}")
            translated = word
        words_ru.append(translated)
    translated_words.append(", ".join(words_ru))

df_topics["words_ru"] = translated_words

df_topics.to_csv(output_ru_path, index=False, encoding="utf-8-sig")

print(f"Перевод завершён и сохранён в {output_ru_path}")

Перевод завершён и сохранён в /content/drive/MyDrive/TextScope/twitter_topics_en_ru.csv


# Просмотр топ-слов по темам (на русском языке)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
output_ru_path = "/content/drive/MyDrive/TextScope/twitter_topics_en_ru.csv"

df = pd.read_csv(output_ru_path)
df

,topic_num,words_en,words_ru
0,-1,"work, get, well, night, home, day, new, change...","работа, получать, хорошо, ночь, дом, день, нов..."
1,0,"work, night, well, weekend, day, good, sleep, ...","работа, ночь, хорошо, выходные, день, хороший,..."
2,1,"study, exam, school, work, project, book, math...","изучать, экзамен, школа, работа, проект, книга..."
3,2,"twitter, video, new, work, blog, picture, twee...","Twitter, видео, новый, работа, блог, картина, ..."
4,3,"cream, ice, cheese, coffee, eat, lunch, tea, d...","крем, лед, сыр, кофе, есть, обед, чай, ужин, п..."
5,4,"climate, change, global, warming, snow, trump,...","климат, изменять, глобальный, потепление, снег..."
6,5,"phone, laptop, iphone, new, computer, internet...","телефон, ноутбук, iPhone, новый, компьютер, Ин..."
7,6,"new, moon, movie, album, dance, watch, trailer...","новый, луна, фильм, альбом, танец, смотреть, т..."
8,7,"ticket, train, flight, plane, york, station, f...","билет, тренироваться, полет, самолет, Йорк, ст..."
9,8,"camp, lake, swim, game, ball, club, pool, lose...","лагерь, озеро, плавать, игра, мяч, клуб, бассе..."
